# Relevant Degreedays
Relevant for Bark Beetle development, considers daylight > 14.5h and minimum temperature above 8.3 degCel

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import os
import re

import xarray as xr
#import cartopy.crs as ccrs

In [2]:
# All directories
root       = "C:/Users/freiste/OneDrive - Ilmatieteen laitos/Documents/IIASA YSSP 2023"
this_dir   =  os.getcwd()

input_dir  = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Original" # to save space, not all data might be there anymore
output_dir = f"{root}/02 - Data/EU/Copernicus_E-OBS_Weather_Postprocessed"

In [3]:
def consider_daylight(lat, year):
    
    # Open daylight hours
    DL = xr.open_dataset(f"{output_dir}/daylight_daily_europe_1980-2022_0.25deg.nc")
    DL.close()
    
    DL = DL.isel(latitude=lat)
    DL = DL.sel(time=year)
    DL = DL.where(DL.daylength >= 14.5, drop=True)
    
    try:
        start = DL.time.values[0]
        end   = DL.time.values[-1]
    except:
        start, end = np.nan, np.nan
    
    return start, end


In [4]:
def relevant_degreedays(yr):
    
    if 1980 <= yr <= 1994:
        strtyr, endyr = 1980, 1994
    elif 1995 <= yr <= 2010:
        strtyr, endyr = 1995, 2010
    elif 2011 <= yr <= 2022:
        strtyr, endyr = 2011, 2022
    else:
        print('Error, year unknown.')
    
    t = xr.open_dataset(f"{input_dir}/tempavg_dailymean_{strtyr}-{endyr}_0.25deg.nc")
    t.close()
       
    
    lats = len(t.latitude)

    res = []
    for ilat in range(lats):

        Temp = t.isel(latitude=ilat)

        # Determine start and end index in time dimensions according to daylength per latitude
        start, end = consider_daylight(ilat, f"{yr}")
        idx1 = Temp.indexes["time"].get_loc(start, method="nearest")
        idx2 = Temp.indexes["time"].get_loc(end,   method="nearest")

        Temp = Temp.tg.isel(time=slice(idx1, idx2+1))
        Temp = Temp.to_dataset(name='relevant_degreedays')

        Temp = Temp.where(8.2 < Temp < 38.9)      # developmental thresholds
        Temp = Temp.sum('time')
        

        Temp = Temp.assign(max_generations = Temp.relevant_degreedays / 557)

        # add data variables start and end of period date
        Temp['season_start'] = start
        Temp['season_end']   = end
        Temp['season_start'] = Temp.season_start.expand_dims(dim={'longitude': 464})
        Temp['season_end']   = Temp.season_end.expand_dims(dim={'longitude': 464})
        Temp['season_start'] = Temp['season_start'].astype('datetime64')
        Temp['season_end']   = Temp['season_end'].astype('datetime64')


        Temp = Temp.assign_coords({'year' : int(yr)})
        Temp = Temp.expand_dims(dim={"year": 1})
        #Temp['relevant_degreedays'] = Temp.where(Temp.relevant_degreedays > 0)

        res.append(Temp)

    res2 = xr.concat(res, dim='latitude')
    res2

    
    return res2


In [5]:
yrs = range(1980,2023)

for y, yr in enumerate(yrs):

    if y==0:
        reldegreedays = relevant_degreedays(yr)
    else:
        reldegreedays = xr.concat([reldegreedays, relevant_degreedays(yr)], dim="year")
    
    print(reldegreedays.year.values)

        
reldegreedays = \
reldegreedays.assign_attrs({'relevant cumulative degree days' : 'Cumulative sum of daily average temperature between April 1st and October 31st for days where there is 14.5 hours of dailight or more (develpmental threshold)'})

[1980]
[1980 1981]
[1980 1981 1982]
[1980 1981 1982 1983]
[1980 1981 1982 1983 1984]
[1980 1981 1982 1983 1984 1985]
[1980 1981 1982 1983 1984 1985 1986]
[1980 1981 1982 1983 1984 1985 1986 1987]
[1980 1981 1982 1983 1984 1985 1986 1987 1988]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997]
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998]
[1980 1

In [6]:
# Export 
reldegreedays.to_netcdf(f"{output_dir}/cumulative_relevant_degreedays_europe_1980-2022_0.25deg.nc")
reldegreedays

C:\Users\freiste\Anaconda3\envs\Zeus\lib\site-packages\pandas\core\arrays\timedeltas.py:728: RuntimeWarning: invalid value encountered in multiply
  return self - (self // other) * other
C:\Users\freiste\Anaconda3\envs\Zeus\lib\site-packages\xarray\coding\times.py:420: RuntimeWarning: invalid value encountered in cast
  int_num = np.array(num, dtype=np.int64)


<xarray.Dataset>
Dimensions:              (latitude: 201, longitude: 464, year: 43)
Coordinates:
  * longitude            (longitude) float64 -40.38 -40.12 ... 75.12 75.38
  * latitude             (latitude) float64 25.38 25.62 25.88 ... 75.12 75.38
  * year                 (year) int64 1980 1981 1982 1983 ... 2020 2021 2022
Data variables:
    relevant_degreedays  (latitude, year, longitude) float32 0.0 0.0 ... 0.0 0.0
    max_generations      (latitude, year, longitude) float32 0.0 0.0 ... 0.0 0.0
    season_start         (latitude, year, longitude) datetime64[ns] NaT ... 2...
    season_end           (latitude, year, longitude) datetime64[ns] NaT ... 2...
Attributes:
    relevant cumulative degree days:  Cumulative sum of daily average tempera...

In [ ]:
precip_longterm.where(2 < precip_longterm < 3, drop=False)

In [ ]:
relevant_degreedays(2020)

In [ ]:
lat, lon = 48.119019, 14.870262

In [ ]:
reldegreedays.sel(latitude = lat,
                  longitude = lon,
                  method='nearest')

In [ ]:
# Development of relevant_degreedays()

t = xr.open_dataset(f"{input_dir}/tempavg_dailymean_1980-1994_0.25deg.nc")
t.close()
yr='1990'

#def BBseason_per_latitude(yr, t=t)

lats = len(t.latitude)

res = []
for ilat in range(lats):

    Temp = t.isel(latitude=ilat)
    
    # Determine start and end index in time dimensions according to daylength per latitude
    start, end = consider_daylight(ilat, yr)
    idx1 = Temp.indexes["time"].get_loc(start, method="nearest")
    idx2 = Temp.indexes["time"].get_loc(end,   method="nearest")

    Temp = Temp.tg.isel(time=slice(idx1, idx2+1)).sum('time')
    Temp = Temp.to_dataset(name='relevant_degreedays')
    
    Temp = Temp.assign(max_generations = Temp.relevant_degreedays / 557)
    
    # add data variables start and end of period date
    #print([start for lon in t.longitude])
    Temp['season_start'] = start
    Temp['season_start'] = Temp.season_start.expand_dims(dim={'longitude': 464})
    Temp['season_end']   = end
    Temp['season_end']   = Temp.season_end.expand_dims(dim={'longitude': 464})
    Temp['season_start'] = Temp['season_start'].astype('datetime64')
    Temp['season_end']   = Temp['season_end'].astype('datetime64')
    #
    #Temp = Temp.assign(season_start = [start for lon in Temp.longitude])

    
    Temp = Temp.assign_coords({'year' : pd.Timestamp(yr).year})
    Temp = Temp.expand_dims(dim={"year": 1})
    #Temp['relevant_degreedays'] = Temp.where(Temp.relevant_degreedays > 0)

    res.append(Temp)

res2 = xr.concat(res, dim='latitude')
res2